In [1]:
# import libraries 
# ----------------
%matplotlib notebook
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt

In [54]:
# check .mat file 
# ---------------
from scipy.io import whosmat
whosmat('Neuhausen_data.mat') # to list variables inside mat file

[('dt', (1, 1), 'double'),
 ('tmax', (1, 1), 'double'),
 ('nt', (1, 1), 'double'),
 ('timevector', (1, 4000), 'double'),
 ('shot', (1, 1), 'double'),
 ('coord_rec', (88, 3), 'double'),
 ('coord_src', (1, 3), 'double'),
 ('distance_rec', (1, 87), 'double'),
 ('distance_src_rec', (1, 88), 'double'),
 ('vx', (4000, 88), 'double'),
 ('vz', (4000, 88), 'double')]

In [67]:
# load .mat file 
# --------------
from scipy.io import loadmat
mat = loadmat('Neuhausen_data.mat')  # to load mat file
# convert them to numpy array
vx = mat['vx']
vz = mat['vz']
dt = mat['dt'][0,0]
nt = mat['nt'][0,0]
tmax = mat['tmax'][0,0]
shot = mat['shot'][0,0]
coord_rec  = mat['coord_rec']
coord_src  = mat['coord_src']
timevector = mat['timevector'][0,:]
distance_rec     = mat['distance_rec'][0,:]
distance_src_rec = mat['distance_src_rec'][0,:]

In [9]:
# initialize parameters 
# ---------------------
alpha = 600  # P-wave velocity
beta = 300   # S-wave velocity

In [84]:
# calculate wavefield gradients 
# -----------------------------
from sympy.calculus import finite_diff_weights # see http://docs.sympy.org/latest/modules/calculus/index.html

# allocate zero gradients
gradx_vx = np.zeros((vx.shape[0],vx.shape[1]))
gradx_vz = np.zeros((vz.shape[0],vz.shape[1]))

# calculate spatial wavefield gradients 
for n in range(1,(vx.shape[1]-1)):
    # FD coefficents according to Fornberg (1988)
    c = finite_diff_weights(2,[-distance_rec[n-1], distance_rec[n] - distance_rec[n], distance_rec[n]],0)
    c = c[-1][-1] # FD weights for 1st derivative
    # horizontal gradient of vx
    gradx_vx[:,n] = ((c[0]*vx[:,n-1]) + (c[1]*vx[:,n]) + (c[2]*vx[:,n+1])) 
    # horizontal gradient of vz
    gradx_vz[:,n] = ((c[0]*vz[:,n-1]) + (c[1]*vz[:,n]) + (c[2]*vz[:,n+1])) 

# P/S separation 
# --------------
# upgoing P-wave on vz
vz_P_up = (1/2.) * (vz + ((beta*((2*beta)/alpha)) * (gradx_vx.cumsum(axis=0)*dt))) 
# upgoing S-wave on vx
vx_S_up = (1/2.) * (vx - ((2*beta) * (gradx_vz.cumsum(axis=0)*dt))) 

# scale data
vx_scal = np.std(vx)
vx_scal = np.tile(vx_scal,(vx.shape[0],1))
vx_plot = vx/vx_scal
vx_S_up_scal = np.std(vx_S_up)
vx_S_up_scal = np.tile(vx_S_up_scal,(vx_S_up.shape[0],1))
vx_S_up_plot = vx_S_up/vx_S_up_scal

In [ ]:
# plotting configuration 
# ----------------------
figure()

# recorded vx
subplot(1,2,1)
set(gcf, 'color', 'w');
set (gca, 'Fontsize', 20);
wigb(vx_plot,distance_src_rec, timevector)
xlabel('Offset (m)')
ylabel('Time (s)')
title('Recorded v_x')
axis([0 80 0.0 0.18])

# P/S separated upgoing vx
subplot(1,2,2)
set(gcf, 'color', 'w');
set (gca, 'Fontsize', 20);
wigb(vx_S_up_plot,distance_src_rec, timevector)
xlabel('Offset (m)')
ylabel('Time (s)')
title('Upgoing S-wave on v_x - filters')
axis([0 80 0.0 0.18])

plt.show()